# Projection of Disparity Net C Result

In [1]:
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
import IO as io
import re
import os
import scipy.ndimage as ndimage

%matplotlib inline

In [2]:
!ls Stereo_SD_d_all/Stereo_SD_d_all_1/

Stereo_SD_d_all_1_CT_cut.stl          Stereo_SD_d_all_1_MASK_right.png
Stereo_SD_d_all_1_CT_cut_coverage.mps dn_c_st3d_1.result.pfm
Stereo_SD_d_all_1_Calibration.txt     dn_c_st3d_1.result.png
Stereo_SD_d_all_1_IMG_REC_left.bmp    dn_cs_st3d_1.result.pfm
Stereo_SD_d_all_1_IMG_REC_right.bmp   dn_cs_st3d_1.result.png
Stereo_SD_d_all_1_IMG_left.bmp        dn_s_st3d_1.result.pfm
Stereo_SD_d_all_1_IMG_right.bmp       dn_s_st3d_1.result.png
Stereo_SD_d_all_1_MASK_REC_eval.png   opencv_bm.result.pfm
Stereo_SD_d_all_1_MASK_REC_left.png   opencv_bm.result.png
Stereo_SD_d_all_1_MASK_REC_right.png  opencv_sgbm.result.pfm
Stereo_SD_d_all_1_MASK_eval.png       opencv_sgbm.result.png
Stereo_SD_d_all_1_MASK_left.png


In [3]:
root_regex = re.compile(r"stereo_sd_d_\w+/Stereo_sd_d_(?P<type>\w+)_(?P<number>\d+)", re.I)

result_address = 'stereo_sd_d_{type}_result/disparity_net_c/Stereo_sd_d_{type}_{number}_result.{format}'

mask_address = '/Stereo_SD_d_{type}_{number}_MASK_REC_eval.png'
left_address = '/Stereo_SD_d_{type}_{number}_IMG_REC_left.bmp'

calibration_regex = re.compile(r"(?P<width>\d+).0{10}\s{2}(?P<height>\d+).0{10}\s{2}(?P<fx>\d+.\d{10})[\s0.]+(?P<cx>\d+.\d{10})[\s0.]+(?P<fy>\d+.\d{10})\s{2}(?P<cy>\d+.\d{10})", re.I)
calibration_address = '/Stereo_SD_d_{type}_{number}_Calibration.txt'

disparity_regex = re.compile(r"dn_c_st3d_1.result.pfm", re.I)


In [4]:
def read_mask(root, type, number):
    return plt.imread(root+mask_address.format(type=type, number=number))

In [5]:
def read_left(root, type, number):
    return plt.imread(root+left_address.format(type=type, number=number))

In [6]:
def read_disparity(root):
    return io.read(root+'/dn_c_st3d_1.result.pfm')

In [7]:
def smoothen_disparity(disparity, kernel=9):
    if kernel==0:
        return disparity
    return ndimage.gaussian_filter(disparity, sigma=kernel)

In [8]:
def read_calibration(root, type, number):
    calibration_file = open(root+calibration_address.format(type=type, number=number), 'r')
    calibrations = [m.groupdict() for m in calibration_regex.finditer(calibration_file.read())]
    for i in range(0,2):
        for key in calibrations[i]:
            calibrations[i][key] = np.float64(calibrations[i][key])
    return calibrations

In [9]:
def create_matrix_q(calibrations):
    baseline = 4
    return np.float32([
        [1, 0, 0, -calibrations[1]['cx']],
        [0, 1, 0, -calibrations[1]['cy']],
        [0, 0, 0, calibrations[1]['fy']],
        [0, 0, -1/baseline, 0]
    ])

In [10]:
def complete_point_cloud(point_cloud, left):
    return np.hstack([-1*point_cloud.reshape(-1, 3), left.reshape(-1, 3)])

In [11]:
def masked_point_cloud(point_cloud, left, mask):
    return np.hstack([np.argwhere(mask == 0), (-1*point_cloud[mask==0]), left[mask==0]])

In [12]:
def remove_outliers(colored_point_cloud, r=(0, 100)):
    if colored_point_cloud.shape[1] == 8:
        z = 4
    else:
        z = 2
    x = colored_point_cloud[:, z]
    x = x[x < r[1]]
    x = x[x > r[0]]

    mean = np.mean(x)
    std = np.std(x)

    up = mean+3*std
    down = mean-3*std
            
    return colored_point_cloud[ (colored_point_cloud[:, z] < up) & (colored_point_cloud[:, z] > down) & ~np.isnan(colored_point_cloud[:, z]) ]


In [13]:
def create_ply(vertices, filename):
    if vertices.shape[1] == 8:
        vertices = vertices.reshape(-1, 8)[:, 2:]
    vertices[:, [0, 1, 2, 3, 4, 5]] = vertices[:, [1, 0, 2, 3, 4, 5]]  # Swapping Columns

    ply_header = '''ply
format ascii 1.0
element vertex %(vert_num)d
property float x
property float y
property float z
property uchar red
property uchar green
property uchar blue
end_header
'''
    with open(filename, 'w') as f:
        f.write(ply_header % dict(vert_num=len(vertices)))
        np.savetxt(f, vertices, '%f %f %f %d %d %d')

In [14]:
def create_xyz(vertices, filename):
    if not vertices.shape[1] == 8:
        return False  # TODO: Replace with proper error handling
    vertices = vertices.reshape(-1, 8)[:, 0:5]
    vertices[:, [0, 1, 2, 3, 4]] = vertices[:, [1, 0, 3, 2, 4]]  # Swapping Columns

    with open(filename, 'w') as f:
        np.savetxt(f, vertices, '%d %d %f %f %f')

In [15]:
for (root, dirs, files) in os.walk('./'):
    for file in files:
        if not re.search(disparity_regex, file):
            continue
        params = re.search(root_regex, root)
        t, n = params.groups()
        mask = read_mask(root, t, n)
        left = read_left(root, t, n)
        disparity = smoothen_disparity(read_disparity(root), kernel=9)
        calibrations = read_calibration(root, t, n)
        q = create_matrix_q(calibrations)
        point_cloud = cv.reprojectImageTo3D(disparity, q)
        c_point_cloud = remove_outliers(complete_point_cloud(point_cloud, left), r=(0, 150))
        m_point_cloud = remove_outliers(masked_point_cloud(point_cloud, left, mask), r=(0, 150))
        create_ply(c_point_cloud, result_address.format(type=t, number=n, format='complete.ply'))
        create_ply(m_point_cloud, result_address.format(type=t, number=n, format='ply'))
        create_xyz(m_point_cloud, result_address.format(type=t, number=n, format='xyz'))
        print(t, n)

distance 5
distance 2
distance 19
distance 17
distance 10
distance 3
distance 4
distance 11
distance 16
distance 20
distance 18
distance 13
distance 14
distance 1
distance 6
distance 8
distance 15
distance 12
distance 9
distance 7
all 30
all 31
all 14
all 13
all 25
all 22
all 23
all 24
all 12
all 15
all 5
all 2
all 3
all 4
all 33
all 34
all 35
all 32
all 10
all 28
all 17
all 21
all 19
all 26
all 18
all 27
all 20
all 29
all 16
all 11
all 1
all 6
all 8
all 9
all 7
smoke 4
smoke 3
smoke 2
smoke 5
smoke 10
smoke 9
smoke 7
smoke 6
smoke 1
smoke 8
angle 14
angle 13
angle 12
angle 15
angle 5
angle 2
angle 3
angle 4
angle 10
angle 17
angle 19
angle 18
angle 20
angle 16
angle 11
angle 8
angle 1
angle 6
angle 7
angle 9
blood 2
blood 3
blood 4
blood 1
